# Bewegte Leiterschleife im Magnetfeld

In [1]:
# Bibliotheken importieren
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt

# Farben definieren
colors = {
    "black" : "#000000",
    "lgrey" : "#f0f0f0",
    "dgrey" : "#c0c0c0",
    "U"     : "#3333cc",
    "I"     : "#e8082c",
    "B"     : "#00d000"
}

# LaTeX-Symbole in Canvas vorzeichnen
img_b = Canvas(width=9, height=12)
img_b.draw_image(Image.from_file("img/dia_b.png"), 0, 0)
img_B = Canvas(width=13, height=17)
img_B.draw_image(Image.from_file("img/dia_B.png"), 0, 0)
img_l = Canvas(width=5, height=12)
img_l.draw_image(Image.from_file("img/dia_l.png"), 0, 0)
img_u_ind = Canvas(width=28, height=12)
img_u_ind.draw_image(Image.from_file("img/dia_u_ind.png"), 0, 0)
img_x = Canvas(width=11, height=9)
img_x.draw_image(Image.from_file("img/dia_x.png"), 0, 0)

In [2]:
class Diagramm():
    
    def __init__(self, lx, ly, layers=None, xlabel=None, ylabel=None):
        self.canvas = MultiCanvas(len(layers), width=lx, height=ly)
        self.c = {}
        for li in range(len(layers)):
            self.c[layers[li]] = self.canvas[li]
        
    def draw_arrow(self, layer, xy, angle, length, color="#000000", line_width=2, tip_width=10):
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)
        with hold_canvas(layer):
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            if length < 15:
                tip_length = 15 + (length - 15)
                width = tip_width / 2 * length / 15
            else:
                tip_length = 15
                width = tip_width / 2
            line_end = (xy[0] + (length - tip_length) * sin_a,
                        xy[1] + (length - tip_length) * cos_a)
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180)
                         )
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180)
                         )
            layer.fill()
    
    def draw_axes(self, layer, xrange, yrange, xlabel=None, ylabel=None, angle=(0,90), margin=(25, 25)):
        self.origin = (margin[0] - xrange[0],
                  margin[1] + yrange[1]
                 )
        # x-Achse
        self.draw_arrow(layer,
                        xy = (self.origin[0] + xrange[0],
                              self.origin[1]),
                        angle = angle[0],
                        length = xrange[1] - xrange[0],
                        line_width = 1,
                        tip_width = 8
                       )
        layer.draw_image(xlabel[0],
                     self.origin[0] + xrange[1] + xlabel[1][0] + 5,
                     self.origin[1] + xlabel[1][1]
                    )
        # y-Achse
        self.draw_arrow(layer,
                        xy = (self.origin[0],
                              self.origin[1] - yrange[0]),
                        angle = angle[1],
                        length = yrange[1] - yrange[0],
                        line_width = 1,
                        tip_width = 8
                       )
        layer.draw_image(ylabel[0],
                     self.origin[0] + ylabel[1][0],
                     self.origin[1] - yrange[1] + ylabel[1][1] - 20
                    )

$$u_{ind} = - \frac{\mathrm{d} \Phi}{\mathrm{d} t} = - \frac{\mathrm{d}A}{\mathrm{d}t} \cdot B = -v \cdot b \cdot B$$

In [4]:
class Leiterschleife(Diagramm):
    
    
    
    def __init__(self):
        layers = [
            "bg",
            "lb",
            "Bb1",
            "Bb2",
            "Bb3",
            "Bb4",
            "Bb5",
            "Bb6",
            "Bb7",
            "Bb8",
            "Bf1",
            "Bf2",
            "Bf3",
            "Bf4",
            "Bf5",
            "Bf6",
            "Bf7",
            "Bf8",
            "lf",
            "ll",
            "laux1",
            "laux2",
            "u_ind_pre",
            "u_ind",
            "u_cursor"
        ]
        
        self.sliders = []
        for s in range(3):
            self.sliders.append(widgets.IntSlider(
                value=0,
                min=-5,
                max=5,
                step=1,
                orientation='vertical',
                description=r"$\color{#00c000}{\vec{B_{%s}} }$" % (s + 1),
                readout=False,
                layout=Layout(width="176px", height="165px"),
                continuous_update=True,
            ))
            self.sliders[s].observe(self.update_fields, "value")
            
        self.l_slider = widgets.IntSlider(min = 10,
                                          max = 100,
                                          value = 40,
                                          readout = False,
                                          description = r"Länge $l$")
        self.l_slider.observe(self.draw_loop, 'value')
        self.b_slider = widgets.IntSlider(min = 10,
                                          max = 50,
                                          value = 30,
                                          readout = False,
                                          description = r"Breite $b$")
        self.b_slider.observe(self.draw_loop, 'value')
        self.v_slider = widgets.IntSlider(min = 1,
                                          max = 3,
                                          value = 3,
                                          readout = False,
                                          description = r"$v$")
        self.v_slider.observe(self.update_v, 'value')
            
        # x-Slider und Play-Button
        self.x_slider = widgets.IntSlider(max = 565,
                                          description = r"$x$")
        self.x_slider.observe(self.move_loop, 'value')
        self.play = widgets.Play(
            value = 0,
            min = 0,
            max = 565,
            step = 1,
            interval = 20,
            description = "Press play",
            disabled = False,
            _repeat = True
        )
        widgets.jslink((self.play, 'value'), (self.x_slider, 'value'))
        
        super().__init__(lx = 750,
                         ly = 460,
                         layers = layers
                        )

        self.lb = Canvas()
        self.lf = Canvas()
        self.draw_background()
        self.last_cursor = self.origin
        self.draw_u()
        self.draw_loop()
        self.draw_fields()
        
        self.sliders[1].value = -5
        
        self.formula = widgets.Label(value=r"$u_{ind} = - \frac{d \Phi}{d t} = \frac{dA}{dt} \cdot B = -v \cdot b \cdot B$")
        
        display(HBox([Box(layout=Layout(width="90px"))] + self.sliders))
        display(self.canvas)
        
        display(HBox([VBox([self.l_slider,
                            self.b_slider,
                            self.v_slider,
                            self.x_slider]),
                      ]))
        display(self.play)
        
    
    
    def draw_background(self):
        
        # Achsen
        self.draw_axes(layer  = self.c["bg"],
                       xrange = (0, 570),
                       xlabel = [img_x, (0, 0)],
                       yrange = (-125, 125),
                       ylabel = [img_u_ind, (-30, 0)],
                       margin = (150, 185),
                      )
        
        # Strichpunktlinie
        ll = self.c["ll"]
        ll.line_width = 1
        ll.set_line_dash([10, 5, 1, 5])
        ll.stroke_style = colors["dgrey"]
        ll.begin_path()
        ll.move_to(0, 80)
        ll.line_to(700, 80)
        ll.stroke()
        
        # Feldgrenzen
        bg = self.c["ll"]        
        for i in range(9):
            bg.line_width = 1
            bg.set_line_dash([1, 3])
            bg.stroke_style = colors["B"]
            bg.begin_path()
            bg.move_to(150 + 60 * i, 80)
            y_to = 185 if i == 0 else 310
            bg.line_to(150 + 60 * i, y_to)
            bg.stroke()            
    
    
    
    def draw_loop(self, *args, **kwarg):

        xy = (125, 80)
        l = self.l_slider.value
        b = self.b_slider.value
        
        with hold_canvas(self.lb):
            self.lb.line_width = 2
            self.lb.stroke_style = colors["black"]
            self.lb.fill_style   = colors["lgrey"]
            
            self.lb.move_to(xy[0], xy[1])
            self.lb.begin_path()
            self.lb.line_to(xy[0], xy[1])
            line_end = (xy[0] + b * 0.25,
                        xy[1] - b * 0.43301270189221935)
            self.lb.line_to(line_end[0], line_end[1])
            self.lb.line_to(line_end[0] - l, line_end[1])
            self.lb.line_to(xy[0] - l, xy[1])
            self.lb.clear()
            self.lb.stroke()
            self.lb.fill()
            
        with hold_canvas(self.lf):
            self.lf.line_width = 2
            self.lf.stroke_style = colors["black"]
            self.lf.fill_style   = colors["lgrey"]
            
            self.lf.move_to(xy[0], xy[1])
            self.lf.begin_path()
            self.lf.line_to(xy[0], xy[1])
            line_end = (xy[0] - b * 0.25,
                        xy[1] + b * 0.43301270189221935)
            self.lf.line_to(line_end[0], line_end[1])
            self.lf.line_to(line_end[0] - l, line_end[1])
            self.lf.line_to(xy[0] - l, xy[1])
            self.lf.clear()
            self.lf.stroke()
            self.lf.fill()
        
        self.move_loop()
        self.draw_u()
    
    
    
    def move_loop(self, *args, **kwarg):
        
        lb    = self.c["lb"]
        lf    = self.c["lf"]
        laux1 = self.c["laux1"]
        laux2 = self.c["laux2"]
        u     = self.c["u_ind"]
        uc    = self.c["u_cursor"]
                
        x = self.x_slider.value - 25
        l = self.l_slider.value
        
        with hold_canvas(lb):
            lb.clear()
            lb.draw_image(self.lb, x + 25, 0)
            
        with hold_canvas(lf):
            lf.clear()
            lf.draw_image(self.lf, x + 25, 0)

        if x > 0:
            
            # Oszilloskop-Spur
            with hold_canvas(u):
                u.stroke_style = colors["U"]
                u.line_width   = 2
                u.begin_path()
                u.move_to(self.last_cursor[0], self.last_cursor[1])
                
                u_new = self.u(x)
                
                if u_new != self.last_cursor[1]:
                    u.line_to(self.origin[0] + x, self.last_cursor[1])
                    
                self.last_cursor = (self.origin[0] + x, self.origin[1] - u_new)
                
                u.line_to(self.last_cursor[0], self.last_cursor[1])
                u.stroke()
            
            # Cursor
            with hold_canvas(uc):
                uc.fill_style  = colors["U"]
                uc.clear()
                uc.fill_arc(self.last_cursor[0], self.last_cursor[1], 3, 0, 2 * pi)
                
            # Hilfslinien
            with hold_canvas(laux1):
                laux1.line_width = 1
                laux1.set_line_dash([1, 3])
                laux1.stroke_style = colors["U"]
                laux1.begin_path()
                laux1.move_to(x + 150, 80)
                laux1.line_to(x + 150, self.origin[1] - u_new)
                laux1.clear()
                laux1.stroke()            
        
        else:
            self.last_cursor = self.origin
            u.clear()
            uc.clear()
            laux1.clear()
            laux2.clear()
            
        if x > l:
            with hold_canvas(laux2):
                laux2.line_width = 1
                laux2.set_line_dash([1, 3])
                laux2.stroke_style = colors["U"]
                laux2.begin_path()
                laux2.move_to(x + 150 - l, 80)
                laux2.line_to(x + 150 - l, self.origin[1])
                laux2.clear()
                laux2.stroke()
    
    
    
    def draw_fields(self):
        
        xy = (150, 80)
        
        self.fields_bg = {}
        self.fields_fg = {}

        l = 60
        
        dx_list = [        [30],
                         [20, 40],
                       [15, 30, 45],
                     [12, 24, 36, 48],
                   [10, 20, 30, 40, 50]
                  ]
        
        for i in range(5):
            
            bg = Canvas()
            bg.line_width = 2
            bg.stroke_style = colors["B"]
            bg.begin_path()
            bgm = Canvas()
                
            fg  = Canvas()            
            fgm = Canvas()
            
            for dx in dx_list[i]:
                
                with hold_canvas(bg):
                    bg.move_to(xy[0] + dx, xy[1])
                    bg.line_to(xy[0] + dx, xy[1] - l)
                    bg.stroke()
                
                bgm.draw_image(bg, 0, l)                    
                
                self.fields_bg[ i + 1] = bgm
                self.fields_fg[-i - 1] = bg
                
                with hold_canvas(fg):
                    self.draw_arrow(fg,
                                    xy = (xy[0] + dx, xy[1]),
                                    angle = 270,
                                    length = l,
                                    color = colors["B"],
                                    tip_width = 6
                                   )                       
                    self.draw_arrow(fgm,
                                    xy = (xy[0] + dx, xy[1]),
                                    angle = 90,
                                    length = l,
                                    color = colors["B"],
                                    tip_width = 6
                                   )               

                self.fields_fg[ i + 1] = fgm
                self.fields_bg[-i - 1] = fg
    
    
    
    def update_fields(self, change):
        
        segment = int(change["owner"].description[-6])
        delta_x = (segment - 1) * 180
        val = self.sliders[segment - 1].value
        
        Bf = self.c["Bf" + str(segment)]
        Bb = self.c["Bb" + str(segment)]
        
        Bf.clear()        
        Bb.clear()
        
        if val != 0:
            
            with hold_canvas(Bb):
                Bb.draw_image(self.fields_bg[val], delta_x)
                
            with hold_canvas(Bf):
                Bf.draw_image(self.fields_fg[val], delta_x)
        
        self.draw_u()
        
    
    
    def draw_u(self, *args, **kwarg):
        
        self.u_array = 125 * [0]
        for s in range(3):
            self.u_array += 60 * [self.sliders[s].value * 10] + 120 * [0]
        self.u_array += 175 * [0]
        
        u = self.c["u_ind_pre"]
        u.set_line_dash([3, 3])
        u.line_width = 1
        u.stroke_style = colors["U"]
        
        u_old = 0

        with hold_canvas(u):
            u.begin_path()
            u.move_to(self.origin[0], self.origin[1])

            for x in range(540):

                u_new = self.u(x)

                # Unstetigkeit berücksichtigen
                if u_new != u_old:
                    u.line_to(self.origin[0] + x, self.origin[1] - u_old)

                u.line_to(self.origin[0] + x, self.origin[1] - u_new)
                u_old = u_new
                u.clear()
                u.stroke()
    
    
    
    def update_v(self, *args, **kwarg):
        self.play.interval = 60 / self.v_slider.value
        self.draw_u()
                
                
    def u(self, x):
        return 0.66666666666666 * self.v_slider.value * self.b_slider.value / self.b_slider.max * (self.u_array[x + 125] - self.u_array[x + 125 - self.l_slider.value])

        


LS = Leiterschleife()

MultiCanvas(height=460, width=750)

Play(value=0, description='Press play', interval=20, max=565)